In [1]:
import numpy as np

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import vgg19
from tensorflow.keras import backend as K

In [4]:
target_path = 'musk_boii.jpg'
style_path = 'mosaic.jpg'

width, height = load_img(target_path).size
image_height = 400
image_width = int(width * image_height/height)

In [5]:
def preprocess_image(image_path):
    
    img = load_img(image_path, target_size = (image_height, image_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    
    return img

In [6]:
def deprocess_image(x):
    
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [8]:
target_image = K.constant(preprocess_image(target_path))
style_ref_image = K.constant(preprocess_image(style_path))
combined_image = K.placeholder((1, image_height, image_width, 3))

input_tensor = K.concatenate([target_image, style_ref_image, combined_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor, weights = 'imagenet', include_top = False)

W1223 18:59:47.339195 140693999523648 deprecation.py:506] From /home/cleanpegasus/.local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [10]:
def gram_matrix(x):
    
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    
    return gram

In [11]:
def style_loss(style, combination):
    
    S = gram_matrix(style)
    C = gram_matrix(combined_image)
    channels = 3
    size = image_height * image_width
    
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * size ** 2)

In [12]:
def total_variation_loss(x):
    
    a = K.square(
                x[:, :image_height - 1, :image_width - 1, :] - 
                x[:, 1:, :img_width - 1, :])
    b = K.square(
                x[:, :image_height - 1, :image_width - 1, :] - 
                x[:, 1:, :img_height - 1, :])
    
    return K.sum(K.pow(a + b, 1.25))

In [13]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1',]
total_variation_weight = 1e-4
style_weight = 1.0
content_weight = 0.025

In [15]:
loss = K.variable(0.0)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss = loss + content_weight + content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    
    layer_features = outputs_dict[layer_name]
    style_ref_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    
    s1 = style_loss(style_ref_features, combination_features)
    
    loss = loss + (style_weight / len(style_layers)) * s1
    
loss = loss + total_variation_weight * total_variation_loss(combined_image)


ValueError: Dimension must be 4 but is 3 for 'transpose_2' (op: 'Transpose') with input shapes: [1,400,400,3], [3].